In [16]:
import os

import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account
def get_big_query(query, credientials_path, project_id = 'trusty-sentinel-283421'):
    
    to_fp = credientials_path
    #get_blob(CONFIG['bucket_name'], CONFIG['bg_api_fp'], to_fp)
    credentials = service_account.Credentials.from_service_account_file(to_fp)
    client = bigquery.Client(credentials= credentials,project=project_id)
    print(f'executing BigQuery: {query}')
    query_job = client.query(query)
    results = query_job.result()
    return results.to_dataframe()

In [11]:
os.listdir("/Users/stevenhansen/GitHub/airflow-docker2/dags/creds/trusty-sentinel-283421-8d4e435c814a.json")

['trusty-sentinel-283421-8d4e435c814a.json', 'monkey.pkl']

In [13]:
credientials_path = "/Users/stevenhansen/GitHub/airflow-docker2/dags/creds/trusty-sentinel-283421-8d4e435c814a.json"

In [14]:
schema = "reddit_db"


In [27]:
q = f'''
SELECT COUNT(*)
FROM {schema}.raw_reddit_pulls
'''
get_big_query(q, credientials_path = credientials_path)

executing BigQuery: 
SELECT COUNT(*)
FROM reddit_db.raw_reddit_pulls



,f0_
0,7828


In [28]:
q = f'''
SELECT COUNT(*)
FROM {schema}.post_exposure
'''
get_big_query(q, credientials_path = credientials_path)

executing BigQuery: 
SELECT COUNT(*)
FROM reddit_db.post_exposure



,f0_
0,7002


In [29]:
q = f'''
SELECT COUNT(*)
FROM {schema}.post_search_terms
'''
get_big_query(q, credientials_path = credientials_path)

executing BigQuery: 
SELECT COUNT(*)
FROM reddit_db.post_search_terms



,f0_
0,7002


In [30]:
q = f'''
SELECT COUNT(*)
FROM {schema}.topic_search_terms
'''
get_big_query(q, credientials_path = credientials_path)

executing BigQuery: 
SELECT COUNT(*)
FROM reddit_db.topic_search_terms



,f0_
0,7002


In [61]:
q = f'''SELECT * 
FROM {schema}.raw_reddit_pulls raw_reddit_pulls
INNER JOIN {schema}.post_exposure post_exposure
  ON raw_reddit_pulls.id = post_exposure.id
INNER JOIN {schema}.post_search_terms post_search_terms
  on raw_reddit_pulls.id = post_search_terms.id
INNER JOIN {schema}.topic_search_terms topic_search
  on raw_reddit_pulls.id = topic_search.id
  '''

df = get_big_query(q, credientials_path = credientials_path)
print(df.shape)
df.head()

executing BigQuery: SELECT * 
FROM reddit_db.raw_reddit_pulls raw_reddit_pulls
INNER JOIN reddit_db.post_exposure post_exposure
  ON raw_reddit_pulls.id = post_exposure.id
INNER JOIN reddit_db.post_search_terms post_search_terms
  on raw_reddit_pulls.id = post_search_terms.id
INNER JOIN reddit_db.topic_search_terms topic_search
  on raw_reddit_pulls.id = topic_search.id
  
(43672, 18)


,id,created,subreddit,subreddit_id,selftext,author_fullname,title,name,url,id_1,num_comments,ups,upvote_ratio,measurement_date,id_2,search_term,id_3,topic
0,163yydd,2023-08-28 16:39:05,Fauxmoi,t5_2rqcm8,,t2_9n3bifg,Cease And Desist sent from Eminem to Republica...,t3_163yydd,https://www.reddit.com/gallery/163yydd,163yydd,227,2718,0.96,202308290207,163yydd,Vivek Ramaswamy,163yydd,us_election_2024
1,163yydd,2023-08-28 16:39:05,Fauxmoi,t5_2rqcm8,,t2_9n3bifg,Cease And Desist sent from Eminem to Republica...,t3_163yydd,https://www.reddit.com/gallery/163yydd,163yydd,227,2718,0.96,202308290207,163yydd,Vivek Ramaswamy,163yydd,us_election_2024
2,163yydd,2023-08-28 16:39:05,Fauxmoi,t5_2rqcm8,,t2_9n3bifg,Cease And Desist sent from Eminem to Republica...,t3_163yydd,https://www.reddit.com/gallery/163yydd,163yydd,227,2718,0.96,202308290207,163yydd,Vivek Ramaswamy,163yydd,us_election_2024
3,163yydd,2023-08-28 16:39:05,Fauxmoi,t5_2rqcm8,,t2_9n3bifg,Cease And Desist sent from Eminem to Republica...,t3_163yydd,https://www.reddit.com/gallery/163yydd,163yydd,227,2718,0.96,202308290207,163yydd,Vivek Ramaswamy,163yydd,us_election_2024
4,163yydd,2023-08-28 16:39:05,Fauxmoi,t5_2rqcm8,,t2_9n3bifg,Cease And Desist sent from Eminem to Republica...,t3_163yydd,https://www.reddit.com/gallery/163yydd,163yydd,227,2727,0.96,202308290207,163yydd,Vivek Ramaswamy,163yydd,us_election_2024


In [62]:
cols = list(df.columns)
cols.remove('ups')
cols.remove('upvote_ratio')
cols.remove('num_comments')


In [63]:
cols

['id',
 'created',
 'subreddit',
 'subreddit_id',
 'selftext',
 'author_fullname',
 'title',
 'name',
 'url',
 'id_1',
 'measurement_date',
 'id_2',
 'search_term',
 'id_3',
 'topic']

In [64]:
df = df[~df[cols].duplicated()]
print(df.shape)

(3501, 18)


In [65]:
df['search_term'].value_counts()

trump                247
Vivek Ramaswamy      245
Asa Hutchinson       243
biden                242
Doug Burgum          242
Florida              240
DeSantis             240
Gov. Ron Desantis    237
Joe Biden            231
Mike Pence           229
Chris Christie       224
Tim Scott            222
trump2024            217
Nikki Haley          201
Donald J. Trump      115
biden2024             99
Glenn Younkin         27
Name: search_term, dtype: int64

In [59]:
search_term_exposure = pd.DataFrame()
for search_term, frame in df.groupby('search_term'):
    search_term_exposure.loc[search_term, 'sum_ups'] = frame.ups.astype(int).sum()
    search_term_exposure.loc[search_term,'sum_comments'] = frame.num_comments.astype(int).sum()
    
print(search_term_exposure.shape)
search_term_exposure.sort_values(by = 'sum_comments',ascending = False)
    

(17, 2)


,sum_ups,sum_comments
Joe Biden,6885096.0,735723.0
trump,7158537.0,699855.0
biden,6375188.0,563534.0
Florida,6634146.0,505239.0
Donald J. Trump,1782787.0,423528.0
DeSantis,5279332.0,394112.0
Mike Pence,4704841.0,260281.0
Chris Christie,1355223.0,113829.0
Gov. Ron Desantis,743400.0,91810.0
Nikki Haley,763420.0,77391.0


## raw_reddit_pulls

In [17]:
q = f'''
SELECT *
FROM {schema}.raw_reddit_pulls
'''

raw_reddit_pulls = get_big_query(q, credientials_path = credientials_path)
print(raw_reddit_pulls.shape)
raw_reddit_pulls.head()

executing BigQuery: 
SELECT *
FROM reddit_db.raw_reddit_pulls

(7828, 9)


,id,created,subreddit,subreddit_id,selftext,author_fullname,title,name,url
0,163yydd,2023-08-28 16:39:05,Fauxmoi,t5_2rqcm8,,t2_9n3bifg,Cease And Desist sent from Eminem to Republica...,t3_163yydd,https://www.reddit.com/gallery/163yydd
1,14pgsah,2023-07-03 06:47:54,politics,t5_2cneq,,t2_hsdekbu,Trump mocked for bursting into bizarre moaning...,t3_14pgsah,https://www.independent.co.uk/news/world/ameri...
2,13qv4sf,2023-05-24 14:28:33,politics,t5_2cneq,,t2_5d6zv,Florida Man Known for Threatening Teachers and...,t3_13qv4sf,https://newrepublic.com/post/172953/florida-ma...
3,163ob5x,2023-08-28 09:55:55,politics,t5_2cneq,,t2_hsdekbu,Trump drops 6 points in post-debate GOP poll,t3_163ob5x,https://thehill.com/homenews/campaign/4174521-...
4,163yr9n,2023-08-28 16:31:46,WhitePeopleTwitter,t5_35n7t,,t2_id1rijny,Eminem just sent a cease-and-desist letter to ...,t3_163yr9n,https://i.redd.it/kt1tq9fa6xkb1.jpg


## post_exposure

In [19]:
q = f'''
SELECT *
FROM {schema}.raw_reddit_pulls
LEFT JOIN 

'''

post_exposure = get_big_query(q, credientials_path = credientials_path)
print(post_exposure.shape)
post_exposure.head()

executing BigQuery: 
SELECT *
FROM reddit_db.post_exposure


(7002, 5)


,id,num_comments,ups,upvote_ratio,measurement_date
0,13ox7hw,17,37,0.73,202308290207
1,15c9zr8,6,43,0.98,202308290207
2,zhgwrb,33,80,0.77,202308290207
3,14z048s,12,2,0.53,202308290207
4,13ow0kt,11,94,0.92,202308290207
